In [1]:
cd vggface

/home/adarsh/celebrity_face_detection/vggface


In [2]:
!python3 setup.py install

running install
/home/adarsh/.local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/home/adarsh/.local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.


In [3]:
!pip install keras_applications==1.0.8
!pip install mtcnn==0.1.0

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle
import cv2
from mtcnn import MTCNN
from PIL import Image

2024-01-31 13:27:35.567696: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 13:27:35.567742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 13:27:35.569730: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 13:27:35.578227: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 13:27:36.797690: W tensorflow/compiler/tf2

In [4]:
cd ..

/home/adarsh/celebrity_face_detection


In [5]:
features_list = np.array(pickle.load(open('embedding.pickle','rb')))
filenames_list = pickle.load(open('filenames.pickle','rb'))

In [6]:
# model = VGGFace(model='resnet50',include_top=False,input_shape=(244,244,3),pooling='avg')
model = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')

In [7]:
detector = MTCNN()

In [9]:
sample_img = cv2.imread('sample/ranbir_dup.png')
results = detector.detect_faces(sample_img)

x ,y ,width ,height = results[0]['box']
face = sample_img[y:y+height,x:x+width]

cv2.imwrite('results/face_image.png', face)

# cv2.imshow('face',face)
# cv2.waitKey(0)

1/1 [==============================] - 0s 143ms/step


True

In [23]:
image = Image.fromarray(face)
image = image.resize((224,224))
face_array = np.asarray(image)
face_array = face_array.astype('float32')
expanded_image = np.expand_dims(face_array ,axis=0)
preprocess_image = preprocess_input(expanded_image)
result = model.predict(preprocess_image).flatten()
print(result)
print(result.shape)

1/1 [==============================] - 1s 1s/step
[0.         0.08629569 0.         ... 5.405649   3.5637383  2.4437695 ]
(2048,)


In [26]:
similarity = []
for i in range(len(features_list)):
    x = cosine_similarity(result.reshape(1,-1) ,features_list[i].reshape(1,-1))[0][0]
    similarity.append(x)
print(len(similarity))

8664


In [31]:
index_pos = sorted(list(enumerate(similarity)) ,reverse=True ,key=lambda x:x[1])[0][0]

temp_img = cv2.imread(filenames_list[index_pos])
cv2.imwrite('results/detected_celeb_image.png', temp_img)

True

In [8]:
def detect_celeb(image_path):
    sample_img = cv2.imread(image_path)
    results = detector.detect_faces(sample_img)

    x ,y ,width ,height = results[0]['box']
    face = sample_img[y:y+height,x:x+width]
    cv2.imwrite('results/face_image.png', face)

    
    image = Image.fromarray(face)
    image = image.resize((224,224))
    face_array = np.asarray(image)
    face_array = face_array.astype('float32')
    expanded_image = np.expand_dims(face_array ,axis=0)
    preprocess_image = preprocess_input(expanded_image)
    result = model.predict(preprocess_image).flatten()
    # print(result)
    # print(result.shape)
    
    similarity = []
    for i in range(len(features_list)):
        x = cosine_similarity(result.reshape(1,-1) ,features_list[i].reshape(1,-1))[0][0]
        similarity.append(x)
    # print(len(similarity))
    
    index_pos = sorted(list(enumerate(similarity)) ,reverse=True ,key=lambda x:x[1])[0][0]

    temp_img = cv2.imread(filenames_list[index_pos])
    cv2.imwrite('results/detected_celeb_image.png', temp_img)
    print('Celeb Detected!')
    

In [9]:
image_path = 'sample/x.png'
detect_celeb(image_path)

1/1 [==============================] - 1s 1s/step
Celeb Detected!
